In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import matplotlib
import matplotlib.cm as cm
import shapefile
from shapely.geometry import shape, mapping, Point, Polygon
from zipfile import ZipFile
from io import BytesIO
import descartes

In [2]:
"""
Looking at population density and distribution of people

Dataset:
Base Zoning: http://opendata.columbus.gov/datasets/96f7642a62f84db997f9e1db4a776995_4
    - Can look at zoning to see which locations are more populated
    
BZA Zoning Variances: http://opendata.columbus.gov/datasets/19786dd084e644a4aea6b33f867dd631_1
    - More Zoning?
    
Recommended Land Use: http://opendata.columbus.gov/datasets/26f0606f94db4c07a63aef3cc8927c9b_21
    - Where we can build charging stations?
    
Population Density Map: https://apps.morpc.org/census2010/

API to convert Lat/Long to census block
    - https://geo.fcc.gov/api/census/#!/area/get_area
    
Conversion for GEOIDs
    - https://www.census.gov/geo/reference/codes/cou.html
    - https://www.census.gov/geo/reference/geoidentifiers.html (General info on how it's generated)
    - https://geoservices.tamu.edu/Services/CensusIntersection/ (Lat/Long to Census block)
        - Current GeoID = STATE+COUNTY+TRACT+BLOCK GROUP = 2+3+6+1=12
    
CountryCode + Lat/Long?
    - https://www.census.gov/geo/maps-data/data/gazetteer2017.html

# TODO: 
- Find the units of block data
- Get better colors for density
- Predicted population through 2019?
- Visualize population density changes over the years?

""";

In [3]:
census_data = pd.read_excel("Data/Columbus_Population.xlsx").iloc[:,0:2]
geoids = census_data.iloc[:,0]
census_data.head()

,GEOID,2010 Total Population
0,390410101003,2258
1,390410102002,1002
2,390410102003,2692
3,390410102004,927
4,390410105201,229


In [4]:
# Credit to http://andrewgaidus.com/Reading_Zipped_Shapefiles/

# Get files necessary for mapping out census blocks
zipFile = ZipFile("Data/ohio_tigerfiles.zip")
filenames = [y for y in sorted(zipFile.namelist()) for ending in ['dbf', 'prj', 'shp', 'shx'] if y.endswith(ending)] 
dbf, prj, shp, shx = [BytesIO(zipFile.read(filename)) for filename in filenames]

reader = shapefile.Reader(shp=shp, shx=shx, dbf=dbf)
attributes, geometry = [], []
field_names = [field[0] for field in reader.fields[1:]]  
for row in reader.shapeRecords():  
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(field_names, row.record)))

In [5]:
# Put tigerfiles nto GeoDataFrame
gdf = gpd.GeoDataFrame(data = attributes, geometry = geometry)[["ALAND10", "GEOID10", "geometry"]]
gdf = gdf.rename(index=str, columns={"ALAND10": "Area", "GEOID10": "GEOID"})
gdf.GEOID = gdf.GEOID.astype(int)
gdf = gdf[gdf["GEOID"].isin(geoids)] # Only get data on GEOIDs that match census data above

# Latitude/Longitude ordering is switched, swap it back and add it as the "geometry" column
block_coord_array = []
for _, row in gdf.iterrows():
    row_coord_array = []
    for coord in mapping(row['geometry'])['coordinates'][0]:
        correct_coord = reversed(coord)
        row_coord_array.append(list(correct_coord))
    
    block_coord_array.append(row_coord_array)

In [6]:
gdf_original = gdf.copy(deep=True)
def pointToGeoid(long, lat):
    type1 = type(long)
    type2 = type(lat)
    assert(type1 == type2), "Parameters must be the same type"
    _pnts = []
    
    if (type1 == list):
        assert(len(long) == len(lat)), "Parameters must have same length"
        for i in range(len(long)):
            _pnts.append(Point(long[i], lat[i]))
    else:
        _pnts.append(Point(long, lat))
        
    pnts = gpd.GeoDataFrame(geometry=_pnts)
    for _, row in gdf_original.iterrows():
        if pnts.within(row.geometry)[0]:
            return row.GEOID
    
    return np.nan

# Returns copy of df with GEOID appended to each long/lat
# Precondition: df is a dataframe
def addGeoidColumn(df):
    assert('long' in df.columns)
    assert('lat' in df.columns)
    df_copy = df.copy(deep=True)
    
    df_copy['GEOID'] = df_copy.apply(lambda x: pointToGeoid(x['long'], x['lat']), axis=1)
    return df_copy
    
testingdf = pd.DataFrame(data=[[-83.039365, 40.023412]], columns=['long', 'lat'])
# GEOID: 390490078112
addGeoidColumn(testingdf)

,long,lat,GEOID
0,-83.039365,40.023412,390490078112


In [7]:
gdf["geometry"] = pd.Series(block_coord_array, index=gdf.index)
gdf.head()

,Area,GEOID,geometry
307,48342325,391590503011,"[[40.243837, -83.400046], [40.243618999999995,..."
308,21876645,391590501003,"[[40.429511999999995, -83.287651], [40.429269,..."
309,13081513,391590502003,"[[40.429511999999995, -83.287651], [40.430355,..."
310,53573385,391590502002,"[[40.351451999999995, -83.35313699999999], [40..."
311,36947189,391590502004,"[[40.346618, -83.319272], [40.346743, -83.3202..."


In [ ]:
### For each GEOID in census_data, get polygon description and area of that block
def getMatchingGEOIDData(geoid):
    return gdf[gdf["GEOID"] == int(geoid)][["Area", "geometry"]]

blockRows = geoids.apply(getMatchingGEOIDData) #An array of DF rows

In [ ]:
### Combine all block data, add it to census_data
block_df = pd.DataFrame()
for row in blockRows:
    block_df = block_df.append(row, ignore_index=True)
    
ohio_population_data = census_data.join(block_df)

In [ ]:
# Now adding county name for each GEOID

county_data = pd.read_excel("Data/Ohio_GEOID_Conversion.xlsx").iloc[:,1:4]

def geoidToCountyLatLong(geoid):
    countyCode = int(geoid / 10000000)
    return county_data[county_data["GEOID"] == countyCode][["NAME"]]

countyRows = geoids.apply(geoidToCountyLatLong)
county_df = pd.DataFrame()
for county in countyRows:
    county_df = county_df.append(county, ignore_index=True)
    
ohio_population_data["CountyNames"] = pd.Series(county_df.NAME.values, index=ohio_population_data.index)

In [ ]:
# Columbus is in Franklin County
grouped_population_data = ohio_population_data.groupby(["CountyNames"])
franklin_county_data = grouped_population_data.get_group("Franklin County")
franklin_county_data.head()

In [ ]:
# Calculate min and max values for density and area of blocks in franklin county
density_array = franklin_county_data["2010 Total Population"] / franklin_county_data["Area"]

fc_max_density = max(density_array)
fc_min_density = min(density_array)
fc_max_area = franklin_county_data.max()["Area"]
fc_min_area = franklin_county_data.min()["Area"]

def areaToMapArea(area):
    # OldRange = (OldMax - OldMin)  
    oldRange = fc_max_area - fc_min_area
    
    # NewRange = (NewMax - NewMin)
    newRange = 1500
    
    # NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin
    return (((area - franklin_county_data.min()["Area (units?)"]) * newRange) / oldRange) + 100

In [ ]:
colors = matplotlib.colors.Normalize(vmin=fc_min_density, vmax=fc_max_density, clip=True)
mapper = cm.ScalarMappable(norm=colors, cmap=cm.cool)

map = folium.Map(
    location=[40.004955, -83.008636],
    zoom_start=12
)

# Returns tuple, first elem is density and is popup information of a given row
def getRowInfo(row):
    row_density = row["2010 Total Population"] / row["Area"]
    popup = 'GEOID: ' + str(row["GEOID"]) + '\n' + 'Density: ' + str(row_density) + '\n' + 'Area: ' + str(row["Area"])
    return (row_density, popup)

# For each row, add a circle in the map
for index, row in franklin_county_data.iterrows():
    ### My laptop can't run anything above ~650, will have to find solution online (cloud)
    if index > 625:
        break
        
    rowInfo = getRowInfo(row)
    
    folium.Polygon(
        locations=row['geometry'],
        fill=True,
        color=matplotlib.colors.rgb2hex(mapper.to_rgba(rowInfo[0])),
        popup=rowInfo[1]
    ).add_to(map)
    
print("DISREGARD UNITS, still need to find out what they are")
map